In [1]:
from peft import LoraConfig, PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    pipeline,
    logging,
)
import transformers
import os

model_id = 'meta-llama/Llama-2-7b-chat-hf'
# model_id = 'codellama/CodeLlama-7b-hf'
# model_id = "meta-llama/Meta-Llama-3-70B-Instruct"
new_model = "/home/hb/dataset_bgp/llm_finetuned/llama2-7b-table135-20split-5k-lorar64"

hf_auth = os.environ.get('hf_token')

bnb_config = transformers.BitsAndBytesConfig(
    load_in_8bit=True,
)

model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    # quantization_config=bnb_config,
    device_map="auto",
    use_auth_token=hf_auth
)

model = PeftModel.from_pretrained(model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

/home/hb/myenv/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
import json
from tablular_eval_util import combine_csv_files, split_dataframe, preprocess_data, run_llm_inference

directory = '/home/hb/dataset_bgp/bgp_tab_dataset_test'
combined_df = combine_csv_files(directory)

# Split the DataFrame into smaller chunks
split_size = 20
data_list = split_dataframe(combined_df, split_size)

# Preprocess the data into the required format
formatted_data = [preprocess_data(chunk) for chunk in data_list]

formatted_data_file = f'llm_table_bgp_data_test_{split_size}.json'
with open(formatted_data_file, 'w') as f:
    json.dump(formatted_data, f, indent=4)

with open(formatted_data_file, 'r') as f:
    formatted_data = json.load(f)

output_results_file = f'table135-20split-2k-with-outputs-{split_size}.json'
run_llm_inference(formatted_data, model, tokenizer, max_length=3050, output_results_file=output_results_file)

Processed 1/10
Processed 2/10


In [3]:
import pandas as pd
import re

output_file = pd.read_json(f"/home/hb/LLM-research/finetune_main/finetuning_tabular/table_read/table135-20split-2k-with-outputs-{split_size}.json")
output_file = output_file["output"]
output_file.to_csv(f"table135-20split-2k-outputs-{split_size}.csv")

timestamps = []

timestamp_pattern = re.compile(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}')

for output in output_file:
    matches = timestamp_pattern.findall(output)
    timestamps.extend(matches)

bgp_llm_output = list(set(timestamps))

print(timestamps)

['2022-03-28 10:15:00', '2022-03-28 08:40:00', '2022-03-28 10:15:00', '2022-03-28 10:15:00', '2022-03-28 10:25:00', '2022-03-28 11:35:00', '2022-03-28 10:25:00', '2022-03-28 11:35:00', '2022-03-28 13:25:00', '2022-03-28 13:25:00', '2022-03-28 14:00:00', '2022-03-28 14:25:00', '2022-03-28 14:00:00', '2022-03-28 14:25:00', '2019-05-08 14:25:00', '2019-05-08 15:15:00', '2019-05-08 14:25:00', '2019-05-08 15:15:00']


In [5]:
import pandas as pd
from tablular_eval_util import combine_csv_files

directory = '/home/hb/dataset_bgp/bgp_tab_dataset_test'
combined_df = combine_csv_files(directory)
combined_df = combined_df[['anomaly_status']]

# Filter the rows with "anomaly detected" in the anomaly_status
filtered_df = combined_df[combined_df['anomaly_status'].str.contains('anomaly detected', na=False)]
# filtered_df.to_csv('/home/hb/dataset_bgp/bgp_tab_dataset_test/test_true_label.csv', index=False)

# Extract the date from the anomaly_status string
true_label = filtered_df['anomaly_status'].str.extract(r'anomaly detected at (\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2})')

# Convert the dates to a list
true_label = true_label[0].tolist()

print(true_label)

['2022-03-28 10:25:00', '2022-03-28 10:30:00', '2022-03-28 10:35:00', '2022-03-28 10:40:00', '2022-03-28 10:55:00', '2022-03-28 11:00:00', '2022-03-28 11:35:00', '2022-03-28 11:40:00', '2022-03-28 11:45:00', '2022-03-28 13:25:00', '2022-03-28 13:30:00', '2022-03-28 13:35:00', '2022-03-28 14:00:00', '2022-03-28 14:05:00', '2022-03-28 14:15:00', '2022-03-28 14:20:00', '2022-03-28 14:25:00', '2022-03-28 15:55:00', '2022-03-28 16:00:00', '2022-03-28 16:05:00', '2019-05-08 15:05:00', '2019-05-08 15:10:00', '2019-05-08 15:15:00', '2019-05-08 15:30:00', '2017-12-12 04:40:00', '2017-12-12 04:45:00', '2017-12-12 04:50:00', '2017-12-12 07:05:00', '2017-12-12 07:10:00']


In [6]:
import pandas as pd
from tablular_eval_util import evaluate_llm_results

evaluation_result = evaluate_llm_results(true_anomalies=true_label, llm_results=bgp_llm_output)
print(f"Evaluation Results {split_size} split:")
print(f"Precision: {evaluation_result['precision']:.2f}")
print(f"Recall: {evaluation_result['recall']:.2f}")
print(f"F1 Score: {evaluation_result['f1_score']:.2f}")
print(f"True Positives: {evaluation_result['true_positives']}")
print(f"False Positives: {evaluation_result['false_positives']}")
print(f"False Negatives: {evaluation_result['false_negatives']}")

Evaluation Results 20 split:
Precision: 0.67
Recall: 0.21
F1 Score: 0.32
True Positives: 6
False Positives: 3
False Negatives: 23
